#### Author: Thejasvi Beleyur
#### Last updated: 2020-03-18

This notebook will detail all the steps required to get matching audio snippets for the annotations of bat flights made for 2018-08-20 between 03:00-04:00 hours. 
This notebook is a copy of 'Matching annotations to audio 2018-08-20 0200-0300.ipynb'. 



### Steps carried out before hand:
1. Generating the video sync for the video file *OrlovaChukaDome_02_20180820_03.00.00-04.00.00[R][@49b6][1]
...avi* with frame-wise light intensities and time stamps

2. Video annotations of the horseshoebats flying done by  Aditya Kanjoor. 


In [1]:
import matplotlib.pyplot as plt
import natsort
import numpy as np
import pandas as pd 
from tqdm import tqdm

package_folder = '../../../bin/'
import os
import sys 
sys.path.append(package_folder) # include the modules in the outer folder in the search path
from process_video_annotations import video_sync_over_annotation_block



In [2]:
%matplotlib notebook

In [3]:
# loading the annotations
annotations = pd.read_csv('annotations/corrected_HBC_video_annotations_Aditya/2018-08-20_0300-0400_Aditya_corrected.csv')

In [4]:
annotations.head()

,video_path,annotation_id,Sampling minute,timestamp from camera,start_timestamp,start_framenumber,end_timestamp,end_framenumber,no. of flying bats,bat behaviour,"strength (1 - strong, 0-weak)","weak in (Behaviour Be, Number Bn)",Comments,Details on the comment,annotator
0,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_1,0,2,2018-08-20 03:00:16,2,2018-08-20 03:00:17,4,1,NaN,1,NaN,NaN,NaN,Aditya
1,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_2,0,2,2018-08-20 03:00:50,11,2018-08-20 03:00:50,22,1,NaN,1,NaN,NaN,NaN,Aditya
2,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_3,0,2,2018-08-20 03:00:51,4,2018-08-20 03:00:53,19,2,NaN,1,NaN,NaN,NaN,Aditya
3,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_4,0,2,2018-08-20 03:00:54,1,2018-08-20 03:00:54,11,1,NaN,1,NaN,NaN,NaN,Aditya
4,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_5,3,2,2018-08-20 03:03:12,22,2018-08-20 03:03:13,7,1,NaN,1,NaN,NaN,NaN,Aditya


In [5]:
annotations.shape

(114, 15)

In [6]:
annots_w_bats = annotations[annotations['no. of flying bats']>0].reset_index(drop=True)

In [7]:
annotations.shape

(114, 15)

In [8]:
annots_w_bats.shape

(114, 15)

In [9]:
annots_w_bats.head()

,video_path,annotation_id,Sampling minute,timestamp from camera,start_timestamp,start_framenumber,end_timestamp,end_framenumber,no. of flying bats,bat behaviour,"strength (1 - strong, 0-weak)","weak in (Behaviour Be, Number Bn)",Comments,Details on the comment,annotator
0,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_1,0,2,2018-08-20 03:00:16,2,2018-08-20 03:00:17,4,1,NaN,1,NaN,NaN,NaN,Aditya
1,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_2,0,2,2018-08-20 03:00:50,11,2018-08-20 03:00:50,22,1,NaN,1,NaN,NaN,NaN,Aditya
2,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_3,0,2,2018-08-20 03:00:51,4,2018-08-20 03:00:53,19,2,NaN,1,NaN,NaN,NaN,Aditya
3,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_4,0,2,2018-08-20 03:00:54,1,2018-08-20 03:00:54,11,1,NaN,1,NaN,NaN,NaN,Aditya
4,OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...,Aditya_2018-08-20_0300-0400_5,3,2,2018-08-20 03:03:12,22,2018-08-20 03:03:13,7,1,NaN,1,NaN,NaN,NaN,Aditya


### Carrying out the common-fps conversion:


In [10]:
# load the video sync file with corrected timestamps
video_sync_file = 'videosync_OrlovaChukaDome_01_20180820_03.00.00-04.00.00[R][@49b5][0].avi_[timestamp_corrected]_[signal_checked].csv'
path_to_videosync = os.path.join('raw_sync','2018-08-19',
                                 video_sync_file)
video_sync = pd.read_csv(path_to_videosync)

In [11]:
video_sync.head()

,Unnamed: 0,frame_number,led_intensity,timestamp,timestamp_verified
0,0,1,0.000100,2018-08-20 03:00:00,2018-08-20 03:00:00
1,1,2,0.000100,2018-08-20 03:00:00,2018-08-20 03:00:00
2,2,3,0.000101,2018-08-20 03:00:00,2018-08-20 03:00:00
3,3,4,0.000100,2018-08-20 03:00:00,2018-08-20 03:00:00
4,4,5,0.000100,2018-08-20 03:00:00,2018-08-20 03:00:00


In [12]:
kwargs = {'timestamp_pattern': '%Y-%m-%d %H:%M:%S'}
kwargs['min_fps']= 21 # Hz
kwargs['min_durn'] = 45.0 # seconds 
kwargs['common_fps'] = 25 # Hz
kwargs['output_folder'] = './common_fps/Aditya_annotations/2018-08-19/'

unprocessed_annotations = []

for i, each_annotation in tqdm(annots_w_bats.iterrows()):
    try:
        success = video_sync_over_annotation_block(each_annotation,video_sync, 
                                                     **kwargs)
    except :
        unprocessed_annotations.append(i)
        print('Unable to process annotation: %s'%annots_w_bats['annotation_id'][i])


0it [00:00, ?it/s]/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use 

Saved annotation :Aditya_2018-08-20_0300-0400_1


../../../bin/process_video_annotations.py:389: UserWarning: Some timestamps below min fps!
  warnings.warn('Some timestamps below min fps!')
../../../bin/process_video_annotations.py:310: UserWarning: FPS fell below the minimum required - ignoring this annotation
  warnings.warn('FPS fell below the minimum required - ignoring this annotation')
../../../bin/process_video_annotations.py:115: UserWarning: could not bring above annotation to common fps,                      please check a) if the corresponding audio file is present                        b) if the ON/OFF of the light signal is clean                           
  ')
2it [00:10,  5.57s/it]

('timestamps below min fps: ', array(['2018-08-20 03:01:31'], dtype=object))
video_path                           OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...
annotation_id                                            Aditya_2018-08-20_0300-0400_2
Sampling minute                                                                      0
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 03:00:50
start_framenumber                                                                   11
end_timestamp                                                      2018-08-20 03:00:50
end_framenumber                                                                     22
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                        

3it [00:14,  5.06s/it]

...

('2018-08-20 03:57:29', '...to...', '2018-08-20 03:58:14')
video_path                           OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...
annotation_id                                          Aditya_2018-08-20_0300-0400_112
Sampling minute                                                                     57
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 03:57:29
start_framenumber                                                                   12
end_timestamp                                                      2018-08-20 03:57:29
end_framenumber                                                                     15
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                                          

113it [08:57,  1.96s/it]

('2018-08-20 03:57:37', '...to...', '2018-08-20 03:58:22')
video_path                           OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...
annotation_id                                          Aditya_2018-08-20_0300-0400_113
Sampling minute                                                                     57
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 03:57:37
start_framenumber                                                                   25
end_timestamp                                                      2018-08-20 03:57:38
end_framenumber                                                                      8
no. of flying bats                                                                   1
bat behaviour                                                                      NaN
strength (1 - strong, 0-weak)                                          

114it [08:59,  1.96s/it]

('2018-08-20 03:57:38', '...to...', '2018-08-20 03:58:23')
video_path                           OrlovaChukaDome_02_20180820_03.00.00-04.00.00[...
annotation_id                                          Aditya_2018-08-20_0300-0400_114
Sampling minute                                                                     57
timestamp from camera                                                                2
start_timestamp                                                    2018-08-20 03:57:38
start_framenumber                                                                   15
end_timestamp                                                      2018-08-20 03:57:38
end_framenumber                                                                     25
no. of flying bats                                                                   2
bat behaviour                                                                        P
strength (1 - strong, 0-weak)                                          

### Matching Audio from the common_fps files

In [13]:
### Finding the matching audio snippet !! 
import glob 
import soundfile as sf
from audio_for_videoannotation import match_video_sync_to_audio, search_for_best_fit

#all_commonfps = glob.glob('common_fps_video_sync*') # get all the relevant common_fps_sync files
all_commonfps = glob.glob('common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_030*')
audio_folder = '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/' # the current folder

# the output folders
audiosync_folder = './sync_audio/'
audioannotation_folder = './annotation_audio/'
fs = 250000 # change according to the recording sampling rate in Hz!! 


In [14]:
all_ccs = []
files_to_run = sorted(all_commonfps)

def generate_matching_audio(each_commonfps, audio_folder, 
                           **kwargs):
    '''
    '''
    print(each_commonfps)
    try:
        video_sync = pd.read_csv(each_commonfps)
        best_audio, syncblock_audio, crosscoef = match_video_sync_to_audio(video_sync, audio_folder, 
                                                                           **kwargs)
        all_ccs.append(crosscoef)
        fname  = os.path.split(each_commonfps)[-1]
        annotation_id = '-'.join(os.path.split(fname)[-1].split('common_fps_video_sync')[-1:])[:-4]

        audiosync_folder = kwargs.get('audiosync_folder', 'sync_audio')
        audioannotation_folder = kwargs.get('audioannotation_folder', 'annotation_audio')

    
        audio_file_base_name = annotation_id+'.WAV'
        sf.write(audiosync_folder+'matching_sync_'+audio_file_base_name, syncblock_audio,fs)
        sf.write(audioannotation_folder+'matching_annotaudio_'+audio_file_base_name, best_audio,fs)
        return(True)
    except:
        print('Could not save ', each_commonfps)
        return(False)



In [15]:
glob.glob(audio_folder+'*.WAV')

['/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000007.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000008.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000009.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000010.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000011.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000012.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/non_spikey_T0000013.WAV',
 '/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/au

In [16]:
natsort.natsorted(os.listdir(audio_folder))

['make_non_spikey_2018-08-20_audio_conversion.py',
 'non_spikey_T0000001.WAV',
 'non_spikey_T0000002.WAV',
 'non_spikey_T0000003.WAV',
 'non_spikey_T0000004.WAV',
 'non_spikey_T0000005.WAV',
 'non_spikey_T0000006.WAV',
 'non_spikey_T0000007.WAV',
 'non_spikey_T0000008.WAV',
 'non_spikey_T0000009.WAV',
 'non_spikey_T0000010.WAV',
 'non_spikey_T0000011.WAV',
 'non_spikey_T0000012.WAV',
 'non_spikey_T0000013.WAV',
 'non_spikey_T0000014.WAV',
 'non_spikey_T0000015.WAV',
 'non_spikey_T0000016.WAV',
 'non_spikey_T0000017.WAV',
 'non_spikey_T0000018.WAV',
 'non_spikey_T0000019.WAV',
 'non_spikey_T0000020.WAV',
 'non_spikey_T0000021.WAV',
 'non_spikey_T0000022.WAV',
 'non_spikey_T0000023.WAV',
 'non_spikey_T0000024.WAV',
 'non_spikey_T0000025.WAV',
 'non_spikey_T0000026.WAV',
 'non_spikey_T0000027.WAV',
 'non_spikey_T0000028.WAV',
 'non_spikey_T0000029.WAV',
 'non_spikey_T0000030.WAV',
 'non_spikey_T0000031.WAV',
 'non_spikey_T0000032.WAV',
 'non_spikey_T0000033.WAV',
 'non_spikey_T0000034.WAV

In [17]:
all_commonfps

['common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_25.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_30.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_13.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_91.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_33.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_71.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_29.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_27.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_38.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400

In [18]:
audio_folder

'/media/tbeleyur/THEJASVI_DATA_BACKUP_3/fieldwork_2018_002/horseshoe_bat/audio_non_spikey/2018-08-19/ch1/'

In [19]:
len(all_commonfps)

49

In [20]:
natsort.natsorted(all_commonfps)

['common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_1.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_6.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_9.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_10.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_13.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_14.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_16.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_18.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_19.csv',
 'common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_23

### A slight change in how annotations are matched to audio. 


In [21]:
sorted_commonfps = natsort.natsorted(all_commonfps)

common_fps_chunks = np.array_split(np.array(sorted_commonfps),4) # split the commonfps files into 10 minutes bins


In [42]:
# get annotation number of first and last annoration for each chunks
for each in common_fps_chunks:
    print(each[0] + ',....' + each[-1] +'\n' )
    first_annot = pd.read_csv(each[0])['timestamp_verified'][0]
    last_annot = pd.read_csv(each[-1])['timestamp_verified']
    print('start: '+first_annot+
          '....End:'+last_annot[last_annot.shape[0]-1]+ '\n'+'\n')

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_1.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_26.csv

start: 2018-08-20 03:00:16....End:2018-08-20 03:15:00


common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_27.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_39.csv

start: 2018-08-20 03:16:50....End:2018-08-20 03:23:28


common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_41.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_73.csv

start: 2018-08-20 03:24:06....End:2018-08-20 03:33:41


common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_74.csv,....common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_92.csv

start: 2018-08-20 03:33:04....End:201

In [32]:
pd.read_csv(each[-1])['timestamp_verified']

'2018-08-20 03:14:15'

In [43]:
file_ranges_for_chunks = [('non_spikey_T0000092','non_spikey_T0000115'),
                          ('non_spikey_T0000105','non_spikey_T0000125'),
                          ('non_spikey_T0000115','non_spikey_T0000134'),
                          ('non_spikey_T0000125','non_spikey_T0000134'),] # the corresponding audio file range for each 10 minute bin


In [44]:
for commonfps_set, file_range in zip(common_fps_chunks, file_ranges_for_chunks):
    for each in commonfps_set:
        generate_matching_audio(each, audio_folder, audio_sync_spikey=False, 
                                file_subset=file_range,
                                audio_fileformat='*.WAV',
                              audiosync_folder='sync_audio/Aditya_annotations/',
                               audioannotation_folder='annotation_audio/Aditya_annotations/')

common_fps/Aditya_annotations/2018-08-19/common_fps_video_syncAditya_2018-08-20_0300-0400_1.csv
('video_fps obtained is :', 25)
Did not find user-provided sample rate - getting it from first file that matches format!
('sampling rate is : ', 250000)
.....finding best audio segment.....


  0%|          | 0/23 [00:00<?, ?it/s]

('max mins for upsampled video', -0.7431066075250137, 1.0)
('max mins', -0.9948413437389053, 1.0)


/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
  4%|▍         | 1/23 [00:05<02:06,  5.76s/it]

('max mins', -1.0, 0.9715677166831985)


...

('max mins', -0.9620530788338345, 1.0)


 89%|████████▉ | 8/9 [00:35<00:04,  4.39s/it]

('max mins', -0.8928210364236441, 1.0)


100%|██████████| 9/9 [00:38<00:00,  4.12s/it]


('Best file pairs are: ', 'non_spikey_T0000132.WAV*non_spikey_T0000133.WAV')
Start and end indices around peak:                  start index: 3452684		          end index: 14952684                  total samples:30015488
('AV Sync was above threshold: ', 0.8614974985172993)


In [58]:
import datetime as dt

In [59]:
print('Notebook saved: ', dt.datetime.now())

('Notebook saved: ', datetime.datetime(2020, 3, 18, 9, 58, 43, 813900))
